In [1]:
import import_ipynb
import sys
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from scipy.interpolate import griddata
from scipy.constants import mu_0

# Importando notebooks diretamente
try:
    from fem_processing import matrices_assembly, gaussian_quadrature
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

Modules were not found: No module named 'fem_processing'


# `errors_and_energy()`

In [ ]:
def errors_and_energy(mesh_data, element_type, uh, analytical_solution):
    error_energy_sum = 0.0  # Reinicia o somatório do erro de energia para este tipo e refinamento
    error_L2_sum = 0.0  # Reinicia o somatório do erro L2 para este tipo e refinamento
    total_energy = 0.0  # Reinicia a energia total para cálculo da capacitância

    # Escolha da quadratura e cálculo de pontos de Gauss com base no tipo do elemento
    type, order = element_type
    if type == 'Triangle':
        gauss_points, weights = gaussian_quadrature.general_triangle_rule(m=7, p=5)
    elif type == 'Quadrangle':
        gauss_points, weights = gaussian_quadrature.square_rule(ng=9)

    for e in range(len(mesh_data['conn_data'])):
        cell = mesh_data['conn_data'][e]
        
        # Coordenadas globais dos nós do elemento
        ai_e, xi_e, yi_e = matrices_assembly.global_nodes_coordinates(e, mesh_data)

        # Solução numérica nos nós do elemento
        uh_e = [uh[node - 1] for node in cell['conn_list']]

        # Get the material properties
        ka_e = cell['stiffness_a_value']

        for xik, wk in zip(gauss_points, weights):
            # Mapeamento isoparamétrico de coordenadas locais (xik) para coordenadas globais (x, y)
            x, y = matrices_assembly.isomapping_to_global_coordinates(ai_e, xik, element_type)

            # Calcula a solução exata e seu gradiente no ponto de Gauss
            u, grad_u = analytical_solution(x, y)

            # Calcula o jacobiano e seu determinante
            Je = matrices_assembly.jacobian(e, mesh_data, element_type, xik) 
            Jdet, Je_inv = np.linalg.det(Je), np.linalg.inv(Je)

            # Gradientes das funções de base no domínio mestre
            varphi, _, _, gradN = matrices_assembly.derivatives_at_master_domain(xik, element_type)

            # Gradientes no domínio físico (transformados pelo jacobiano)
            grad_phi = Je_inv @ gradN

            # Gradiente da solução aproximada u_h no ponto e Gradiente do erro 
            grad_uh = np.dot(grad_phi, uh_e)

            # Interpolação da solução aproximada u_h no ponto
            u_fem = np.dot(uh_e, varphi)

            # Calcula o erro quadrático L2 no ponto de Gauss
            error_squared = (u - u_fem) ** 2

            # Calcula o erro quadrático de energia no ponto de Gauss
            grad_error_squared = np.linalg.norm(grad_u - grad_uh) ** 2

            # Energia local no ponto de Gauss
            local_energy = 1/2 * ka_e * np.linalg.norm(grad_uh) ** 2

            # Acumula os erros L2 e de energia
            error_L2_sum += wk * error_squared * abs(Jdet)
            error_energy_sum += wk * grad_error_squared * abs(Jdet)
            total_energy += wk * local_energy * abs(Jdet)

    return np.sqrt(error_L2_sum), np.sqrt(error_energy_sum), total_energy

# `magnetic_density_field()`

In [ ]:
def magnetic_field_density(FINITE_ELEMENT, mesh_data, uh):
    """
    Calcula o vetor densidade de campo magnético B em todo o domínio
    com base nos valores nodais do potencial magnético A_z.

    Parameters:
    mesh_data: dict
        Dados da malha, incluindo conectividade e propriedades materiais.
    element_type: tuple
        Tipo do elemento e ordem do elemento (ex.: ('Triangle', 1)).
    uh: array
        Valores da solução aproximada (potencial vetor magnético) nos nós.

    Returns:
    B_domain: list of arrays
        Vetor densidade de campo magnético \( \mathbf{B} \) para cada elemento.
    """
    conn = [element['conn_list'] for element in mesh_data['conn_data']]
    B_domain, x_coords, y_coords, Bx, By = [], [], [], [], []

    type, order = FINITE_ELEMENT
    try:
        if type == 'Triangle' and order in [1]:
            gauss_points, _ = gaussian_quadrature.general_triangle_rule(m=7, p=5)
        elif type == 'Triangle' and order in [2, 3]:
            gauss_points, _ = gaussian_quadrature.general_triangle_rule(m=7, p=5)
        elif type == 'Quadrangle':
            gauss_points, _ = gaussian_quadrature.square_rule(ng=9)
        else:
            raise ValueError("Tipo de elemento ou ordem não suportados.")
    except Exception as e:
        raise RuntimeError(f"Erro na seleção de quadratura: {e}")

    for e in range(len(conn)):
        uh_e = [uh[node - 1] for node in conn[e]]
        B_gauss = []

        for xik in gauss_points:
            Je = matrices_assembly.jacobian(e, mesh_data, FINITE_ELEMENT, xik)
            Jdet, Je_inv = np.linalg.det(Je), np.linalg.inv(Je)
            if np.abs(Jdet) < 1e-12:
                raise ValueError(f"Jacobiano com determinante próximo de zero no elemento {e}.")
            _, _, _, gradN = matrices_assembly.derivatives_at_master_domain(xik, FINITE_ELEMENT)
            grad_phi = Je_inv @ gradN
            grad_uh = np.dot(grad_phi, uh_e)

            B_x, B_y = grad_uh[1], -grad_uh[0]
            B_gauss.append([B_x, B_y])
            x_coords.append(xik[0])
            y_coords.append(xik[1])
            Bx.append(B_x)
            By.append(B_y)

        B_domain.append(B_gauss)

    x_coords, y_coords = np.array(x_coords), np.array(y_coords)
    Bx, By = np.array(Bx), np.array(By)

    # Plot do campo vetorial
    plt.figure(figsize=(10, 8))
    plt.quiver(x_coords, y_coords, Bx, By, scale=1, scale_units='xy', angles='xy', color='blue', alpha=0.7)
    plt.title("Campo Magnético \( \\mathbf{B} \)", fontsize=16)
    plt.xlabel("x", fontsize=14)
    plt.ylabel("y", fontsize=14)
    plt.axis('equal')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()

    return B_domain

# `electric_field_intensity()`

In [ ]:
def electric_field_intensity(element_type, mesh_data, uh):
    """
    Calcula as componentes do campo elétrico, normaliza e plota o campo vetorial.

    Parâmetros:
    -----------
    mesh_data : dict
        Dados da malha contendo coordenadas, conectividade, etc.
    element_type : tuple
        Tipo e ordem do elemento (e.g., ('Triangle', 1)).
    uh : array
        Solução aproximada nos nós da malha.
    plot : bool
        Indica se o campo elétrico deve ser plotado. Default é True.

    Retorno:
    --------
    electric_field : list
        Lista contendo as componentes do campo elétrico para cada elemento em cada ponto de Gauss.
    """
    electric_field = []  # Lista para armazenar as componentes do campo elétrico
    quiver_data = []  # Lista para coordenadas e vetores para o plot

    # Escolha da quadratura com base no tipo do elemento
    type, order = element_type
    
    # Get the Gauss points and weights
    if type == 'Triangle' and order in [1]:
        gauss_points, _ = gaussian_quadrature.general_triangle_rule(m=1, p=1)
    elif type == 'Triangle' and order in [2, 3]:
        gauss_points, _ = gaussian_quadrature.general_triangle_rule(m=7, p=5)
    elif type == 'Quadrangle':
        gauss_points, _ = gaussian_quadrature.square_rule(ng=9)

    for e in range(len(mesh_data['conn_data'])):
        cell = mesh_data['conn_data'][e]
        
        # Coordenadas globais dos nós do elemento
        ai_e, xi_e, yi_e = matrices_assembly.global_nodes_coordinates(e, mesh_data)

        # Solução numérica nos nós do elemento
        uh_e = [uh[node - 1] for node in cell['conn_list']]

        for xik in gauss_points:
            # Mapeamento isoparamétrico de coordenadas locais para globais
            x, y = matrices_assembly.isomapping_to_global_coordinates(ai_e, xik, element_type)

            # Calcula o jacobiano e seu determinante
            Je = matrices_assembly.jacobian(e, mesh_data, element_type, xik) 
            Jdet, Je_inv = np.linalg.det(Je), np.linalg.inv(Je)

            # Gradientes das funções de base no domínio mestre
            _, _, _, gradN = matrices_assembly.derivatives_at_master_domain(xik, element_type)

            # Gradientes no domínio físico (transformados pelo jacobiano)
            grad_phi = Je_inv @ gradN

            # Gradiente da solução aproximada u_h no ponto de Gauss (campo elétrico)
            grad_uh = np.dot(grad_phi, uh_e)

            # Componentes do campo elétrico
            Ex, Ey = -grad_uh[0], -grad_uh[1]

            # Calcula a magnitude do vetor elétrico
            magnitude = np.sqrt(Ex**2 + Ey**2)

            # Normaliza o campo elétrico
            Ex_norm = Ex / magnitude if magnitude != 0 else 0
            Ey_norm = Ey / magnitude if magnitude != 0 else 0

            # Adiciona os dados para o gráfico
            quiver_data.append((x, y, Ex_norm, Ey_norm))

            # Armazena o campo elétrico calculado neste ponto
            electric_field.append((x, y, Ex_norm, Ey_norm))

    # Extraindo coordenadas e vetores
    x_vals, y_vals, Ex_vals, Ey_vals = zip(*quiver_data)

    # Plot do campo elétrico normalizado
    plt.figure(figsize=(8, 6))
    plt.quiver(
        x_vals, y_vals, Ex_vals, Ey_vals,
        angles='xy', scale_units='xy', scale=1E4, color='blue'
    )
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Campo Elétrico Normalizado')
    plt.axis('equal')
    plt.grid(True)
    plt.show()

# `electromagnetic_field_intensity()`

In [ ]:
def electromagnetic_field_intensity(element_type, mesh_data, uh, kc2, plot=True):
    """
    Calcula as componentes do campo elétrico na eletrodinâmica, normaliza e plota o campo vetorial.

    Parâmetros:
    -----------
    mesh_data : dict
        Dados da malha contendo coordenadas, conectividade, etc.
    element_type : tuple
        Tipo e ordem do elemento (e.g., ('Triangle', 1)).
    uh : array
        Solução aproximada nos nós da malha (representa H_z).
    omega : float
        Frequência angular.
    mu : float
        Permeabilidade magnética do meio.
    kc_squared : float
        Constante de propagação ao quadrado (\( k_c^2 \)).
    plot : bool, optional
        Indica se o campo elétrico deve ser plotado. Default é True.

    Retorno:
    --------
    electromagnetic_field : list
        Lista contendo as componentes do campo elétrico para cada elemento em cada ponto de Gauss.
    """
    electromagnetic_field = []  # Lista para armazenar as componentes do campo elétrico
    quiver_data = []  # Lista para coordenadas e vetores para o plot
    omega = 2*np.pi*1E6
    mu = mu_0

    # Escolha da quadratura com base no tipo do elemento
    type, order = element_type

    # Get the Gauss points and weights
    if type == 'Triangle' and order in [1]:
        gauss_points, _ = gaussian_quadrature.general_triangle_rule(m=3, p=2)
    elif type == 'Triangle' and order in [2, 3]:
        gauss_points, _ = gaussian_quadrature.general_triangle_rule(m=7, p=5)
    elif type == 'Quadrangle':
        gauss_points, _ = gaussian_quadrature.square_rule(ng=9)

    for e in range(len(mesh_data['conn_data'])):
        cell = mesh_data['conn_data'][e]

        # Coordenadas globais dos nós do elemento
        ai_e, xi_e, yi_e = matrices_assembly.global_nodes_coordinates(e, mesh_data)

        # Solução numérica nos nós do elemento
        uh_e = [uh[node - 1] for node in cell['conn_list']]

        for xik in gauss_points:
            # Mapeamento isoparamétrico de coordenadas locais para globais
            x, y = matrices_assembly.isomapping_to_global_coordinates(ai_e, xik, element_type)

            # Calcula o jacobiano e seu determinante
            Je = matrices_assembly.jacobian(e, mesh_data, element_type, xik)
            Jdet, Je_inv = np.linalg.det(Je), np.linalg.inv(Je)

            # Gradientes das funções de base no domínio mestre
            _, _, _, gradN = matrices_assembly.derivatives_at_master_domain(xik, element_type)

            # Gradientes no domínio físico (transformados pelo jacobiano)
            grad_phi = Je_inv @ gradN

            # Gradiente da solução aproximada u_h (H_z) no ponto de Gauss
            grad_uh = np.dot(grad_phi, uh_e)

            # Componentes do campo elétrico baseadas nas equações fornecidas
            Ex = (-1j * omega * mu / kc2) * grad_uh[1]  # Derivada parcial de H_z em relação a y
            Ey = (1j * omega * mu / kc2) * grad_uh[0]   # Derivada parcial de H_z em relação a x

            # Armazena o campo elétrico calculado neste ponto
            electromagnetic_field.append((x, y, Ex, Ey))

    # Plot do campo elétrico normalizado (se habilitado)
    if plot:
        x_vals, y_vals, Ex_vals, Ey_vals = zip(*quiver_data)
        plt.figure(figsize=(8, 6))
        plt.quiver(
            x_vals, y_vals, Ex_vals, Ey_vals,
            angles='xy', scale_units='xy', scale=1E4, color='blue'
        )
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Campo Elétrico Normalizado')
        plt.axis('equal')
        plt.grid(True)
        plt.show()

